# Database Query Checker
Scratchpad for testing queries on the database

In [2]:
from pprint import pprint
import pandas as pd
from RecommenderAgent import WhiskyRecommender
import sqlite3
recommender = WhiskyRecommender()
alchemy_path = recommender.alchemy
sqlite_path = recommender.dbpath
kw_path = recommender.kwpath
con_to_db = recommender.dbcon

Database files available. Agent initialising.
Agent loading complete.


In [3]:
term = "ph"
col="Name"
con = con_to_db()
cur = con.cursor()
cur.execute(
            
        )
x = cur.fetchall()
con.close()
x

TypeError: function takes at least 1 argument (0 given)